In [1]:

from glob import glob
from tqdm import tqdm
#from rdkit import Chem
import numpy as np
import pandas as pd
import re
import shutil
import os

In [2]:
os.getcwd()

'/Users/bangseongjin/Desktop/2022SamsungAIchallenge/Codes'

In [3]:
!gdown "https://drive.google.com/file/d/1UhL-lkwilxMrJtW1tPoA0Nra3LPNGRTK"

zsh:1: command not found: gdown


In [4]:
train = pd.read_csv("../data/train_set.ReorgE.csv")
train.columns = ["index", "SMILES", "Reorg_g", "Reorg_ex"]
print(len(train))
train.head()

18157


,index,SMILES,Reorg_g,Reorg_ex
0,train_0,CC[C@H]1CCCCN1C(=O)[C@@H](C)OC(=O)c1c(C)oc(-n2...,0.631486,0.535060
1,train_1,O[C@@H](CNC1CC1)CN1CCc2sccc2C1,0.825901,1.116781
2,train_2,N#CCCNC(=O)[C@@]1(O)CCSC1,1.463943,0.964848
3,train_3,COC[C@H]1CN(c2ccc(OCC[C@@H](C)O)cc2)C(=O)O1,0.166669,0.161458
4,train_4,N#Cc1c(-c2ccccc2OCC(N)=O)[nH]c(C(N)=O)c1N,0.313820,0.338862


In [5]:
os.chdir('/Users/bangseongjin/Desktop/2022SamsungAIchallenge/Codes')

In [6]:
os.getcwd()

'/Users/bangseongjin/Desktop/2022SamsungAIchallenge/Codes'

In [7]:
if os.path.isdir("ex_file"):
    pass
else:
    os.mkdir("ex_file")

if os.path.isdir("g_file"):
    pass
  
else:
    os.mkdir("g_file")

train_mol = sorted(glob("../data/mol_files/train_set/*.mol"))

if not os.path.isfile("ex_file/train_0_ex.csv"):
    for i in tqdm(train_mol):
        result = []
        tmp = open(i, 'r').read().split("\n")
        for j in tmp:
            tmp_ = re.sub(' +', ' ', j.lstrip()).split(' ')
            if len(tmp_) > 11:
                result.append(tmp_)
            
        file_name = i.split("/")[-1].split('.')[0]
    
        if "ex" in file_name:
            pd.DataFrame(result).to_csv(f"ex_file/{file_name}" + ".csv", index = False)
        elif "g" in file_name:
            pd.DataFrame(result).to_csv(f"g_file/{file_name}" + ".csv", index = False)

In [8]:
max_len = 0
error = []
g_data, ex_data = [], []
mol = []

for i in tqdm(train[train.columns[0]]):
    tmp_g = pd.read_csv(f"g_file/{i}" + '_g.csv')

    if len(tmp_g) >= max_len:
        max_len = len(tmp_g)
        
    for j in tmp_g["3"]:
        mol.append(j)


100%|██████████| 18157/18157 [00:19<00:00, 915.86it/s]


In [9]:
print(tmp_g)
print(max_len)

          0        1       2  3  4  5  6  7  8  9  10  11  12  13  14  15
0    0.6476   0.2797  0.0464  C  0  0  0  0  0  0   0   0   0   0   0   0
1    2.0299   0.0820  0.1534  O  0  0  0  0  0  0   0   0   0   0   0   0
2    2.5425   0.3247  1.4442  C  0  0  0  0  0  0   0   0   0   0   0   0
3    4.0451   0.0793  1.4018  C  0  0  0  0  0  0   0   0   0   0   0   0
4    4.7851   0.2133  2.7560  C  0  0  2  0  0  0   0   0   0   0   0   0
..      ...      ...     ... .. .. .. .. .. .. ..  ..  ..  ..  ..  ..  ..
107  5.0195  16.2097 -0.5201  H  0  0  0  0  0  0   0   0   0   0   0   0
108  5.3465  13.7915 -0.3138  H  0  0  0  0  0  0   0   0   0   0   0   0
109  6.6508  12.1604  0.6959  H  0  0  0  0  0  0   0   0   0   0   0   0
110  7.0070   9.7198  0.8650  H  0  0  0  0  0  0   0   0   0   0   0   0
111  5.1314   2.9900  1.9569  H  0  0  0  0  0  0   0   0   0   0   0   0

[112 rows x 16 columns]
310


In [10]:
for i in range(10):
  print(mol[i])

C
C
C
C
C
C
C
N
C
O


In [11]:
mol_list = list(set(mol))
g_data, ex_data = [], []

def get_mol(data):
    if data in mol_list:
        return mol_list.index(data) + 1
    
    else:
        return -1

for i in tqdm(train[train.columns[0]]):
    tmp_g = pd.read_csv(f"g_file/{i}" + '_g.csv').iloc[:, :4]
    tmp_ex = pd.read_csv(f"ex_file/{i}" +'_ex.csv').iloc[:, :4]
    
    tmp_g["3"] = tmp_g["3"].apply(lambda x : get_mol(x)) # apply는 인자값(get_mol)에 따라 각 셀을 변화시키는 함수이다.
    tmp_ex["3"] = tmp_ex["3"].apply(lambda x : get_mol(x))
 
  
    tmp_g = np.array(tmp_g)
    tmp_ex = np.array(tmp_ex)
    
    if max_len != len(tmp_g):
        # tmp_g : 좌표정보, 원소
        # [[0] * 4] * max_len-tmp_g.shape[0] --> [[0],[0],[0],[0]] * max_len-tmp_g.shape[0]
        tmp_g = np.concatenate((tmp_g, np.array([[0] * 4] * (max_len-tmp_g.shape[0]))))
        tmp_ex = np.concatenate((tmp_ex, np.array([[0] * 4] * (max_len - tmp_ex.shape[0]))))
    elif max_len == len(tmp_g):
        tmp_g = tmp_g
        tmp_ex = tmp_ex
    
    g_data.append(tmp_g)
    ex_data.append(tmp_ex)


100%|██████████| 18157/18157 [00:55<00:00, 328.34it/s]


In [12]:

print(g_data[0])
print(type(g_data))

[[ 2.8972 -2.3256  0.4309  1.    ]
 [ 3.3467 -1.0151  1.0796  1.    ]
 [ 3.3365  0.2041  0.1293  1.    ]
 ...
 [ 0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.    ]]
<class 'list'>


In [13]:

g_data = np.array(g_data).reshape(len(g_data), -1) # -1 의 의미는 자동적으로 배열 할당
print(g_data)

ex_data = np.array(ex_data).reshape(len(ex_data), -1)

train_x = np.concatenate((g_data, ex_data), axis = 1)
train_y = np.array(train.loc[:, ["Reorg_g", "Reorg_ex"]])

print(train_x.shape)
print(train_y.shape) # 트레이닝 데이터 준비 끝

[[ 2.8972 -2.3256  0.4309 ...  0.      0.      0.    ]
 [ 2.2446  2.0206 -0.4231 ...  0.      0.      0.    ]
 [ 4.8695 -1.9497 -5.1308 ...  0.      0.      0.    ]
 ...
 [ 1.423  -0.1053  0.7295 ...  0.      0.      0.    ]
 [11.486   1.9573  5.8098 ...  0.      0.      0.    ]
 [ 0.6476  0.2797  0.0464 ...  0.      0.      0.    ]]
(18157, 2480)
(18157, 2)


In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.svm import SVR

In [ ]:
#LR = MultiOutputRegressor(lgb.LGBMRegressor(random_state=42, n_estimators=1000, max_depth=61, learning_rate=0.075)).fit(train_x, train_y)

SV_g = SVR(kernel = 'linear', C = 1.0, epsilon = 0.1)
SV_ex = SVR(kernel = 'linear', C = 1.0, epsilon = 0.1)
SV_g.fit(train_x, train_y[:,0])
SV_ex.fit(train_x, train_y[:,1])

#차원 맞춰주어야함
'''
document
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
'''

In [34]:
os.getcwd()
os.chdir('/Users/bangseongjin/Desktop/2022SamsungAIchallenge/data')

In [35]:
if os.path.isdir("test_ex_file"):
    pass
else:
    os.mkdir("test_ex_file")

if os.path.isdir("test_g_file"):
    pass
else:
    os.mkdir("test_g_file")

test_mol = sorted(glob("mol_files/test_set/*.mol"))
if not os.path.isdir("test_ex_file/test_0_ex.csv"):
    for i in tqdm(test_mol):
        result = []
        tmp = open(i, 'r').read().split("\n")
        for j in tmp:
            tmp_ = re.sub(' +', ' ', j.lstrip()).split(' ')
            if len(tmp_) > 11:
                result.append(tmp_)

        file_name = i.split('/')[-1].split('.')[0]

        if "ex" in file_name:
            pd.DataFrame(result).to_csv(f"test_ex_file/{file_name}" + ".csv", index = False)
        elif "g" in file_name:
            pd.DataFrame(result).to_csv(f"test_g_file/{file_name}" + ".csv", index = False)

100%|██████████| 914/914 [00:01<00:00, 591.12it/s]


In [32]:
#절대 경로로 변경
os.getcwd()
os.chdir('/Users/bangseongjin/Desktop/2022SamsungAIchallenge/Codes')

In [36]:
test = pd.read_csv("test_set.csv")

test_g, test_ex = [], []

for i in tqdm(test[test.columns[0]]):
    tmp_g = pd.read_csv(f"test_g_file/{i}" + '_g.csv').iloc[:, :4]
    tmp_ex = pd.read_csv(f"test_ex_file/{i}" +'_ex.csv').iloc[:, :4]
    
    tmp_g["3"] = tmp_g["3"].apply(lambda x : get_mol(x))
    tmp_ex["3"] = tmp_ex["3"].apply(lambda x : get_mol(x))
    
    tmp_g = np.array(tmp_g)
    tmp_ex = np.array(tmp_ex)
    
    if len(tmp_g) < max_len:
        tmp_g = np.concatenate((tmp_g, np.array([[0] * 4] * (max_len-tmp_g.shape[0]))))
        tmp_ex = np.concatenate((tmp_ex, np.array([[0] * 4] * (max_len - tmp_ex.shape[0]))))
    elif len(tmp_g) == max_len:
        pass
    else:
        tmp_g = tmp_g[:max_len]
        tmp_ex = tmp_ex[:max_len]
    
    test_g.append(tmp_g)
    test_ex.append(tmp_ex)

100%|██████████| 457/457 [00:01<00:00, 296.59it/s]


In [37]:
test_g = np.array(test_g).reshape(len(test_g), -1)
test_ex = np.array(test_ex).reshape(len(test_ex), -1)

test_x = np.concatenate((test_g, test_ex), axis = 1)

print(test_x.shape)

(457, 2480)


In [38]:
pred = SV.predict(test_x)

In [39]:
submission = pd.read_csv("sample_submission.csv")

submission.head()

,index,Reorg_g,Reorg_ex
0,test_0,0,0
1,test_1,0,0
2,test_2,0,0
3,test_3,0,0
4,test_4,0,0


In [50]:
os.getcwd()
if not os.path.isdir("/Users/bangseongjin/Desktop/2022SamsungAIchallenge/submission_file"):
    os.mkdir('/Users/bangseongjin/Desktop/2022SamsungAIchallenge/submission_file')

In [53]:

submission.loc[:, ["Reorg_g", "Reorg_ex"]] = pred
submission.to_csv('/Users/bangseongjin/Desktop/2022SamsungAIchallenge/submission_file/submission_SVR.csv', index = False)


'/'